# Hydraulisen järjestelmän data-analyysi

Tämä notebook sisältää hydraulisen järjestelmän datan generoinnin ja visualisoinnin. Notebook on jaettu kahteen pääosaan:
1. Datan generointi
2. Datan visualisointi ja analyysi

## Tarvittavat kirjastot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import os

# Asetetaan matplotlib asetukset
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['font.size'] = 12

## 1. Datan generointi

### 1.1 Apufunktiot datan generointiin

Määritellään funktiot, jotka generoivat eri tilojen dataa:

In [ ]:
def generate_normal_digging(n_samples):
    """Generoi normaalin kaivuutilanteen dataa
    
    Parametrit:
    n_samples (int): Generoitavien näytteiden määrä
    
    Palauttaa:
    tuple: (pumpun_ohjaus, paine)
    """
    pumpun_ohjaus = np.random.normal(60, 7, n_samples)
    paine = np.random.normal(160, 15, n_samples)
    
    # Rajataan arvot järkeviin rajoihin
    pumpun_ohjaus = np.clip(pumpun_ohjaus, 45, 76)
    paine = np.clip(paine, 120, 205)
    
    return pumpun_ohjaus, paine

def generate_normal_empty(n_samples):
    """Generoi tyhjän kauhan liikkeiden dataa"""
    pumpun_ohjaus = np.random.normal(28, 8, n_samples)
    paine = np.random.normal(38, 8, n_samples)
    
    pumpun_ohjaus = np.clip(pumpun_ohjaus, 18, 43)
    paine = np.clip(paine, 18, 56)
    
    return pumpun_ohjaus, paine

def generate_normal_full(n_samples):
    """Generoi täyden kauhan liikkeiden dataa"""
    pumpun_ohjaus = np.random.normal(82, 7, n_samples)
    paine = np.random.normal(217, 18, n_samples)
    
    pumpun_ohjaus = np.clip(pumpun_ohjaus, 67, 98)
    paine = np.clip(paine, 180, 270)
    
    return pumpun_ohjaus, paine

def generate_overload(n_samples):
    """Generoi ylikuormitustilanteen dataa"""
    pumpun_ohjaus = np.random.normal(94, 2, n_samples)
    paine = np.random.normal(337, 20, n_samples)
    
    pumpun_ohjaus = np.clip(pumpun_ohjaus, 73, 101)
    paine = np.clip(paine, 280, 432)
    
    return pumpun_ohjaus, paine

def generate_hose_break(n_samples):
    """Generoi letkurikkotilanteen dataa
    
    Letkurikossa paine romahtaa äkillisesti, mutta pumpun ohjaus
    voi vaihdella paljon ennen kuin vika havaitaan.
    """
    pumpun_ohjaus = np.random.normal(75, 18, n_samples)
    paine = np.random.normal(15, 30, n_samples)
    
    pumpun_ohjaus = np.clip(pumpun_ohjaus, 30, 97)
    paine = np.clip(paine, 0, 266)
    
    return pumpun_ohjaus, paine

### 1.2 Datan generointi ja tallennus

Generoidaan data kaikille tiloille ja yhdistetään ne yhdeksi dataframeksi:

In [ ]:
# Määritellään näytemäärät eri tiloille
samples = {
    'normal_digging': 3685,  # Yleisin tila
    'normal_empty': 1110,    # Tyhjän kauhan liikkeet
    'normal_full': 1205,     # Täyden kauhan liikkeet
    'overload': 3000,        # Ylikuormitustilanteet
    'hose_break': 1000       # Letkurikot
}

# Generoidaan data jokaiselle tilalle
data = []
for state, n_samples in samples.items():
    # Kutsutaan oikeaa generointifunktiota tilan mukaan
    if state == 'normal_digging':
        pumpun_ohjaus, paine = generate_normal_digging(n_samples)
    elif state == 'normal_empty':
        pumpun_ohjaus, paine = generate_normal_empty(n_samples)
    elif state == 'normal_full':
        pumpun_ohjaus, paine = generate_normal_full(n_samples)
    elif state == 'overload':
        pumpun_ohjaus, paine = generate_overload(n_samples)
    else:  # hose_break
        pumpun_ohjaus, paine = generate_hose_break(n_samples)
    
    # Lisätään data listaan
    data.append(pd.DataFrame({
        'pumpControl': pumpun_ohjaus,
        'pressure': paine,
        'state': state
    }))

# Yhdistetään kaikki datat
df = pd.concat(data, ignore_index=True)

# Tallennetaan data CSV-tiedostoon
os.makedirs('../Data', exist_ok=True)
df.to_csv('../Data/hydraulic_data.csv', index=False)

print("Data generoitu ja tallennettu!")
print("\nDatan muoto:", df.shape)
print("\nTilojen jakauma:")
print(df['state'].value_counts())

## 2. Datan visualisointi ja analyysi

### 2.1 Datan lataus ja väripaletti

In [ ]:
# Ladataan data
df = pd.read_csv('../Data/hydraulic_data.csv')

# Määritellään väripaletti eri tiloille
palette = {
    'normal_digging': '#2ecc71',  # Vihreä
    'normal_empty': '#3498db',     # Sininen
    'normal_full': '#f1c40f',      # Keltainen
    'overload': '#e74c3c',         # Punainen
    'hose_break': '#9b59b6'        # Violetti
}

### 2.2 Visualisointien luonti

Luodaan neljä kuvaajaa:
1. Scatter plot pumpun ohjauksen ja paineen suhteesta
2. Pie chart tilojen jakaumasta
3. Density plot pumpun ohjauksesta
4. Density plot paineesta

In [ ]:
# Luodaan kuva, jossa on 4 alakuvaajaa
fig = plt.figure(figsize=(20, 15))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# 1. Scatter plot
ax1 = fig.add_subplot(gs[0, 0])
for state in df['state'].unique():
    mask = df['state'] == state
    ax1.scatter(df[mask]['pumpControl'], df[mask]['pressure'], 
                alpha=0.5, label=state, c=[palette[state]])

ax1.set_xlabel('Pumpun ohjaus (%)')
ax1.set_ylabel('Paine (bar)')
ax1.set_title('Pumpun ohjauksen ja paineen suhde eri tiloissa')
ax1.legend()
ax1.grid(True)

# 2. Pie chart
ax2 = fig.add_subplot(gs[0, 1])
counts = df['state'].value_counts()
ax2.pie(counts, 
        labels=[f"{state}\n{count} kpl\n({count/len(df)*100:.1f}%)" 
                for state, count in counts.items()],
        colors=[palette[state] for state in counts.index],
        autopct='')
ax2.set_title('Tilojen jakauma')

# 3. Density plot - pumpun ohjaus
ax3 = fig.add_subplot(gs[1, 0])
for state in df['state'].unique():
    sns.kdeplot(data=df[df['state']==state], x='pumpControl',
                label=state, color=palette[state], ax=ax3)
ax3.set_xlabel('Pumpun ohjaus (%)')
ax3.set_ylabel('Tiheys')
ax3.set_title('Pumpun ohjauksen jakauma eri tiloissa')

# 4. Density plot - paine
ax4 = fig.add_subplot(gs[1, 1])
for state in df['state'].unique():
    sns.kdeplot(data=df[df['state']==state], x='pressure',
                label=state, color=palette[state], ax=ax4)
ax4.set_xlabel('Paine (bar)')
ax4.set_ylabel('Tiheys')
ax4.set_title('Paineen jakauma eri tiloissa')

# Tallennetaan kuva
os.makedirs('../Visualization', exist_ok=True)
plt.savefig('../Visualization/hydraulic_analysis.png', dpi=300, bbox_inches='tight')
plt.close()

### 2.3 Tilastollinen analyysi

Lasketaan keskeiset tilastolliset tunnusluvut jokaiselle tilalle:

In [ ]:
# Tulostetaan tilojen lukumäärät ja prosenttiosuudet
print("Lukumäärät ja prosenttiosuudet:")
for state, count in df['state'].value_counts().items():
    print(f"{state}: {count} kpl ({count/len(df)*100:.1f}%)")

print("\nPumpun ohjauksen tilastot:")
print(df.groupby('state')['pumpControl'].describe())

print("\nPaineen tilastot:")
print(df.groupby('state')['pressure'].describe())